# EJERCICIOS

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras import Sequential
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Concatenate
from keras.datasets import cifar10
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam

2025-01-21 03:39:15.531906: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-21 03:39:15.539227: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-21 03:39:15.548360: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-21 03:39:15.551007: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-21 03:39:15.558024: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Verifica si TensorFlow detecta la GPU
if tf.config.list_physical_devices('GPU'):
    print("¡GPU encontrada!")
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
else:
    print("¡GPU no encontrada!")

¡GPU encontrada!
Num GPUs Available:  1


I0000 00:00:1737427172.068714  106988 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1737427172.095668  106988 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1737427172.095885  106988 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


## EJERCICIO 1


Supongamos que queremos predecir dos objetivos diferentes (por ejemplo, precio y categoría) a partir de dos conjuntos de características diferentes (por ejemplo, características visuales y características de texto de un producto).

* Características visuales: un conjunto de 128 características numéricas, que podrían representar, por ejemplo, valores de píxeles o características extraídas de imágenes.
* Características de texto: un conjunto de 256 características numéricas, representando posiblemente la codificación de texto o características lingüísticas.
* Precio (Salida 1): un valor numérico que representa el precio de un producto.
* Categoría (Salida 2): una etiqueta de categoría, que asumiremos que puede tomar 10 valores diferentes (por ejemplo, 10 categorías diferentes de productos).

Voy a generar datos aleatorios

In [4]:
# Generación de datos ficticios
num_muestras = 1000  # Número de muestras en el conjunto de datos

# Características visuales: 128 características numéricas (x1)
caracteristicas_visuales = np.random.rand(num_muestras, 128)

# Características de texto: 256 características numéricas (x2)
caracteristicas_texto = np.random.rand(num_muestras, 256)

# Precio (Salida 1): Valor numérico (por ejemplo, precio de un producto) (y1)
precio = np.random.rand(num_muestras, 1)

# Categoría (Salida 2): 10 categorías posibles (codificadas en one-hot) y2
categorias = np.random.randint(0, 10, size=(num_muestras, 1))
categorias_one_hot = tf.keras.utils.to_categorical(categorias, num_classes=10)

Divide los datos en entrenamiento y prueba

In [19]:

x_temp1, x_test1, y_temp1, y_test1 = train_test_split(caracteristicas_visuales, precio, test_size=0.2, random_state=17)


x_train1, x_val1, y_train1, y_val1 = train_test_split(x_temp1, y_temp1, test_size=0.2, random_state=17)


x_temp2, x_test2, y_temp2, y_test2 = train_test_split(caracteristicas_texto, categorias_one_hot, test_size=0.2, random_state=17)


x_train2, x_val2, y_train2, y_val2 = train_test_split(x_temp2, y_temp2, test_size=0.2, random_state=17)

In [20]:
print("x_train1:",x_train1.shape)
print("x_val1:",x_val1.shape)
print("x_test1:",x_test1.shape)

print("y_train1:",y_train1.shape)
print("y_val1:",y_val1.shape)
print("y_test1:",y_test1.shape)

print("-------------------------------------------")

print("x_train2:",x_train2.shape)
print("x_val2:",x_val2.shape)
print("x_test2:",x_test2.shape)

print("y_train2:",y_train2.shape)
print("y_val2:",y_val2.shape)
print("y_test2:",y_test2.shape)

x_train1: (640, 128)
x_val1: (160, 128)
x_test1: (200, 128)
y_train1: (640, 1)
y_val1: (160, 1)
y_test1: (200, 1)
-------------------------------------------
x_train2: (640, 256)
x_val2: (160, 256)
x_test2: (200, 256)
y_train2: (640, 10)
y_val2: (160, 10)
y_test2: (200, 10)


Construye el modelo:
* Crea dos ramas de entrada, una para cada tipo de características.
* Las ramas se fusionarán para predecir el precio.
* Una de las ramas se utilizará también para predecir la categoria.

El esquema es el siguiente:

<img src="./esquema_modelo_funcional.png" alt="esquema modelo funcional"/>

In [26]:
tf.keras.backend.clear_session()
tf.random.set_seed(17)

input_layer_visual = tf.keras.layers.Input(shape=[128]) 
input_layer_texto = tf.keras.layers.Input(shape=[256])  
norm_layer_visual = tf.keras.layers.Normalization(axis=-1)  # Normalizar sobre la última dimensión
norm_layer_texto = tf.keras.layers.Normalization(axis=-1)  # Normalizar sobre la última dimensión


norm_visual = norm_layer_visual(input_layer_visual)
norm_texto = norm_layer_texto(input_layer_texto)
hidden_visual = tf.keras.layers.Dense(64, activation="relu")(norm_visual)
hidden_texto = tf.keras.layers.Dense(64, activation="relu")(norm_texto)
concat = tf.keras.layers.concatenate([hidden_visual, hidden_texto])
output_categorico = tf.keras.layers.Dense(10)(hidden_texto)
output_precio = tf.keras.layers.Dense(1)(concat)

model = tf.keras.Model(inputs=[input_layer_visual, input_layer_texto], outputs=[output_categorico, output_precio])

I0000 00:00:1737430626.764905  106988 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1737430626.765106  106988 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1737430626.765187  106988 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1737430626.820055  106988 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Comprueba creando una imagen del modelo que es igual que el que se pide

In [ ]:
tf.keras.utils.plot_model(model, "resultados4_7/modelo_ejer1.png", show_shapes=True)

Compila el modelo especificando las pérdidas y métricas para cada salida (utiliza el optimizador Adam)

Entrena el modelo 

Evalúa el modelo en el conjunt de pruebas

## EJERCICIO 2

El objetivo de este ejercicio es entrenar un modelo en un subconjunto de clases (Modelo A) y luego usar este modelo para entrenar otro modelo en un subconjunto diferente de clases (Modelo B), primero sin y luego con el conocimiento transferido del Modelo A.

Carga los datos de CIFAR-10, representa alguna de sus imágenes con sus etiquetas, haz una lista con las etiquetas que tiene (busca en internet) y normaliza los datos dividiendo entre 255.0.

Divide los datos en conjuntos de entrenamiento, pruebas y validación para el modelo A y para el modelo B:
* El modelo A utiliza las clases "avión", "automóvil", "pájaro" y "gato"
* El modelo B utiliza las clases "ciervo", "perro", "rana" y "caballo" 

Entrena el Modelo A

Entrenar el Modelo B desde 0

Entrenar el Modelo B utilizando las capas aprendidas del Modelo A (excepto la capa de salida)

Evaluar los dos modelos para el conjunto de datos B en el conjunto de pruebas

¿En este caso merece la pena?